# MATH 405/607 

# Numerical Methods for Differential Equations

[[Instructor: Christoph Ortner]](http://www.math.ubc.ca/~ortner/)  [[CANVAS]](https://canvas.ubc.ca/courses/55324)



# Interpolation and Quadrature 

* polynomials 
* review taylor expansion 
* polynomial interpolation 
* splines
* quadrature 
* composite quadrature 
* Newton-Cotes quadrature 

### Literature 

* [Driscoll, Fundamentals of Numerical Computations](https://fncbook.github.io/fnc/localapprox/overview.html)
* Süli and Mayers, An Introduction to Numerical Analysis, Ch 6-11 : range of elementary and advanced material
* Trefethen, Approximation Theory and Approximation Practise : more advanced material

In [ ]:
include("../math405.jl")

### Approximation by "simple functions"

Virtually all of numerical analysis and scientific computing relies on our ability to approximate "complex" functions by "simple" functions. For example, it is easy to define functions $f$ through 

$$\begin{aligned}
    & f(x) = {\rm erf}(x) = \int_{0}^x \exp( - t^2) \, dt, \text{  or}   \\ 
    & x^2 f_xx + x f_x + (x^2 - \alpha^2) f = 0, 
\end{aligned}$$

but it is not so easy to evaluate them. This is particularly common when solving inhomogeneous and nonlinear differential equations. We often know that there is a unique solution specified by the DE, but we are unable to evaluate it because we cannot find a (or there is no) closed-form expression.

The idea is to *approximate* $f(x)$ by *simple functions*.

### Approximation by "simple functions"

What are simple functions? 

Functions we can evaluate on a computer:

*LAYER 1:* only allow +, -, *, $\div$
  - Polynomials: $p(x) = a_0 + a_1 x + \dots + a_N x^N$ 
  - Rationals: $r(x) = p(x) / q(x)$

*LAYER 2:* Exponentials and special functions
  - e.g., $\exp, \sin, \cos, \log, \arccos, \arcsin, \tan, \arctan$, bessel, $\Gamma$, $B$, erf, airy, $\zeta$, ... 
  - These are normally reduced to LAYER 1, usually rationals, and there are excellent software libraries to evaluate them. In Julia see e.g. `SpecialFunctions.jl`

A numerical analyst should use information about the problem at hand to decide which of these function classes are suitable. In this lecture, for simplicity, we will focus on polynomials.

## Polynomials 

**Definition:** a function $p : \mathbb{R} \to \mathbb{R}$ of the form 
$$ 
    p(x) = a_0 + a_1 x + \dots a_N x^N
$$
with $a_n \in \mathbb{R}$ and $a_N \neq 0$ is a polynomial of degree $N$. The $a_i$ are called the coefficients. 

(We can also consider complex polynomials but for this lecture we will focus on the real case.)

A polynomial given in the above format (there are many others!) is typically  evaluated using the Horner scheme: 

$$
    p(x) = a_0 + x(a_1 + x (a_2 + \dots + x (a_{N-1} + x a_N)
$$

In julia this is implemented as [`evalpoly`](https://github.com/JuliaLang/julia/blob/f130d9baa88e353601672c2627b275d02c1f308c/base/math.jl#L135)

In [ ]:
# 1 + x + x^2 + ... + x^9 = (x^10 - 1)/(x-1)
x = 0.54321
@show evalpoly(x, ones(10))
@show (x^(10) - 1) / (x-1);

**Weierstrass Approximation Theorem:** 
* Let $f \in C([a, b])$ then for all $\epsilon > 0$ there exists a polynomial $p$ such that $\| f - p \|_\infty \leq \epsilon$. 
* Or equivalently: the polynomials are dense in $C[a,b]$ for all $a < b \in \mathbb{R}$. 

**Proof:** Using Bernstein polynomials. See recorded lecture. 

In particular this proves that smooth functions are dense in $C[a, b]$.

## Polynomial Interpolation 

We now know that polynomials are dense. Obvious questions that arise are how to construct good approximations in practise, e.g., given $f$ how can we construct the best approximation of degree $N$? This is difficult and we won't discuss it here. 

A typical scenario is that one has function values $f(x_n)$ available at some points $x_n \in [a, b]$, $n = 0, \dots, N$. The idea then is to construct a polynomial $p$ matching $f$ at those points. This is called *polynomial interpolation* and the points $x_n$ are called the *interpolation nodes*.

Which degree should we use? 

A degree $N$ polynomial has $N+1$ parameters so most likely we can match exactly $N+1$ values.

In fact we have the following result: 

**Proposition:** For $N+1$ distinct points $x_0, \dots, x_N$ and values $f_0, \dots, f_N$ there exists a unique polynomial $p$ of degree $N$ such that $p(x_n) = f_n$ for $n = 0, \dots, N$. 

**Proof Existence:** constructive
 $$
     L_n(x) := \frac{\prod_{k \neq n} (x - x_k)}{\prod_{k \neq n} (x_n - x_k)}
 $$
 then $L_n(x_k) = \delta_{nk}$ and therefore 
 $$ 
     p(x) = \sum_{n = 0}^N f_n L_n(x)
 $$
 

 **Proof uniqueness:** WLOG $f_n = 0$. If $p$ has degree $N$ then 
 $$
     p(x) = a_N \prod_{n = 1}^N (x - x_n)
 $$
 But $p(x_0) = 0$ implies $a_N = 0$. 

### Interpolation - the wrong way 

Solve the linear system for the coefficients $a_i$:
$$ 
    \sum_{n = 0}^N a_i x_i^n = f_i, \qquad i = 0, \dots, N
$$

Because there is a unique interpolant, and a polynomial is uniquely determined by its coefficients, it follows that there is a unique solution. The system matrix $V = ( x_i^n )_{i, n = 0}^N$ is called *Vandermonde matrix*.

In [ ]:
"compute the coefficients of the interpolating polynomial"
get_coeffs(X, F) =  [ x^n for x in X, n=0:length(X) ] \ F

f = x -> cos(pi*x)
N = 3
X = range(0, 1, length=N+1)
a = get_coeffs(X, f.(X))

xp = range(0, 1, length=300)
plot(xp, f.(xp), lw=2, label = "f", size=(500, 350))
plot!(xp, evalpoly.(xp, Ref(a)), lw=2, label = "p$(N)")
scatter!(X, f.(X), ms=8, c =  2, label = "")
title!("||f-p||_∞ ≈ $(norm(f.(xp)-evalpoly.(xp, Ref(a)), Inf))")

### Beware the Witch of Agnesi
[some historical context](https://mathworld.wolfram.com/WitchofAgnesi.html)

In [ ]:
f = x -> 1 / (1 + 10 * x^2)
plt = plot(f, -1, 1, lw=2, label = "f", size=(500, 350), legend = nothing)

In [ ]:
myevalpoly(xp::AbstractVector, f::Function, X) = evalpoly.(xp, Ref(get_coeffs(X, f.(X))))

f = x -> 1 / (1 + 10 * x^2)

xp = range(-1, 1, length=500)
plt = plot(xp, f.(xp), lw=2, label = "f", size=(700, 500), legend = :top)
for N in [5, 10, 15, 20]
    plot!(xp, myevalpoly(xp, f, range(-1,1,length=N)), lw=2, label = "p$(N)")
end
plt

### Chebyshev Interpolation 

It is initially counter-intuitive but it turns out that equispaced interpolation is a poor choice for global polynomial approximation. A much better choice are the Chebyshev nodes
$$ 
   x_n := \cos( \pi n / N )
$$
where - for convenience - we are now approximating functions on $f \in C([-1,1])$.

In [ ]:
MATH405.chebyshev_projection(8)

In [ ]:
chebnodes(N) = cos.(range(0, pi, length=N+1))
f = x -> 1 / (1 + 10 * x^2)

xp = range(-1, 1, length=500)
plt = plot(xp, f.(xp), lw=2, label = "f", size=(500, 300), legend = :topright)
for N in [5, 10, 15, 20]
    pN = myevalpoly(xp, f, chebnodes(N)); err = norm(f.(xp)-pN, Inf)
    plot!(xp, pN, lw=2, label = "p$(N), err = $(round(err, digits=5))")
end
plt

Understanding this effect goes beyond this introductory lecture, but I hope we can return to it later. For now we take these examples as a warning that *global* polynomial interpolation is fraught with traps but they are well understood and can be avoided. A very nice book to study this is [Nick Trefethen's Approximation Theory and Approximation Practise](https://people.maths.ox.ac.uk/trefethen/ATAP/ATAPfirst6chapters.pdf)

### Interpolation Error Analysis 

Interpolation of smooth functions generally leads to small errors. "The smoother the function is the faster the rate of convergence."

**Theorem:** Let $f \in C^{N+1}([x_0, x_N])$ and $p_N(x)$ the interpolant on the nodes $x_0 < \dots < x_N$. Then, for every $x \in [x_0, x_N]$ there exists $\xi = \xi(x)$ s.t. 
$$
    f(x) - p_N(x) 
    = 
    (x - x_0) (x - x_1) \cdots (x - x_N) \frac{f^{(N+1)}(\xi)}{(N+1)!}.
$$

**Corollary:** If the $a \leq x_0 < \dots < x_N \leq b$ then 
$$ 
    \| f - p_N\|_{L^\infty(a,b))}
    \leq 
    \frac{(b - a)^{N+1} \| f^{(N+1)} \|_{L^\infty(a,b)}}{(N+1)!}.
$$

But a better estimate would be 
$$ 
    \| f - p_N\|_{L^\infty(a,b))}
    \leq 
    \|\ell\|_{L^\infty(a,b)} \frac{\| f^{(N+1)} \|_{L^\infty(a,b)}}{(N+1)!}.
$$
where $\ell(x) = \prod_{n=0}^N (x-x_n)$. 

For example, if the nodes are symmetric about the mid-point of the interval then one can prove  (cf. exercises)
$$
    \| \ell \|_{L^\infty} \leq \Big(\frac{b - a}{2}\Big)^{N+1}
$$

In [ ]:
eval_L(x, X) = prod( (x - xn) for xn in X )
norm_L(X) = maximum( abs( eval_L(x, X) ) for x in range(-1, 1, length=10_000) )
NN = 6:6:60
nrm_equi = [ norm_L(range(-1, 1, length=N+1)) for N in NN ]
nrm_cheb = [ norm_L(chebnodes(N)) for N in NN ]
plot(NN, nrm_equi, lw=3, label = "equispaced", size = (400, 200))
plot!(NN, nrm_cheb, lw=3, label = "chebyshev", yscale = :log10)

**Corollary:** If the $a \leq x_0 < \dots < x_N \leq b$ and the nodes are symmetric about $(a+b)/2$ then 
$$ 
    \| f - p_N\|_{L^\infty(a,b))}
    \leq 
    \Big(\frac{b - a}{2}\Big)^{N+1} \frac{\| f^{(N+1)} \|_{L^\infty(a,b)}}{(N+1)!}.
$$

The following numerical results don't quite follow from what we covered above, but are closely related. More 
material to be covered when we do spectral methods at the end.

In [ ]:
# Convergence for a C^{2,1} function: (with a weaker notion of 3rd derivative!)
f3 = x -> abs(x)^3
xe = range(-1,1,length=10_000)
NN3 = (2).^(2:8)
err3 = [ norm( myevalpoly(xe, f3, chebnodes(N)) - f3.(xe), Inf ) for N in NN3 ]
plot(NN3, err3, lw=3, label = "error-f3", m=:o, ms=4, yscale = :log10, xscale = :log10, size = (400, 200))
plot!(NN3[2:4], 3*NN3[2:4].^(-3), lw=2, ls=:dash, c=:black, label = L"N^{-3}")

In [ ]:
# Convergence for an analytic function:
fa = x ->  1 / (1+4*x^2)
NNa = 6:6:60
erra = [ norm( myevalpoly(xe, fa, chebnodes(N)) - fa.(xe), Inf ) for N in NNa ]
plot(NNa, erra, lw=3, label = "error-agnesi", m=:o, ms=4, c=2, yscale = :log10, size = (400, 200))
plot!(NN3, err3, lw=4, label =  "error-agnesi-f3", m=:o, ms=4, c=1, xlims = [5, 65])
plot!(NNa[2:5], 5*exp.(-1/2*NNa[2:5]), lw=2, ls=:dash, c=:black, label = L"e^{- N / 2}")

There are ways to deal with this numerical instability, e.g., barycentric interpolation (see assignment A2); lsq, irlsq, ... 

## Splines 

Motivation:
$$ 
    \| f - p_N\|_{L^\infty(a,b))}
    \leq 
    \Big(\frac{b - a}{2}\Big)^{N+1} \frac{\| f^{(N+1)} \|_{L^\infty(a,b)}}{(N+1)!}.
$$

Apart from the (extremely complex) choice of nodes we have two parameters to play with: the degree $N$ and the interval-length $(b-a)$. We have already seen that increasing $N$ must be done carefully. By contrast, making $b-a$ smaller appears to be very robust from the above formula! 

But the domain is given to us, we do not get to choose it. Instead what we can do is sub-divide it and perform independent approximations on each subdomain. This leads to so-called *spline approximations*.

### Piecewise linear splines 

Subdivide $[a, b]$ into $a = x_0 < x_1 < \dots, < x_M = b$ and on $(x_{m-1}, x_m)$ approximate
$$
    f(x) \approx s_1(x) := f(x_{m-1}) \frac{x_m - x}{x_m - x_{m-1}} 
                + f(x_m) \frac{x - x_{m-1}}{x_m - x_{m-1}}
$$

In [ ]:
f = x -> (1 + 10 * x^2)^(-1)
X = range(-1, 1, length = 9)
plot(f, -1, 1, lw=3, label = L"f", size = (600, 300))
plot!(X, f.(X), lw=3, marker = :o, ms = 6, label = L"s_1", c=3)

#### Piecewise quadratic splines 

On each subinterval $[x_{m-1}, x_m]$ let $s_2(x) \in \mathcal{P}_2$ be the nodal interpolant at the nodes 
$$
x_{m-1}, \qquad  \frac{1}{2}(x_{m-1}+x_m), \qquad x_m
$$ 
i.e. at end-points and mid-point. This is more tedious to write down explicitly but can be done; just use the lagrange basis functions $L_0, L_1, L_2$ on each sub-interval.

In [ ]:
f = x -> 1/(1+10*x^2)
p2(x, f, x0, x1, x2) = ( f(x0) * (x-x1)*(x-x2)/(x0-x1)/(x0-x2)
                        +f(x1) * (x-x0)*(x-x2)/(x1-x0)/(x1-x2)
                        +f(x2) * (x-x0)*(x-x1)/(x2-x0)/(x2-x1) )
X = range(-1, 1, length=9)
Xm = (X[2:end] + X[1:end-1])/2
plt = plot(f, -1, 1, lw=3, label = L"f", size = (600, 300))
plot!(X, f.(X), lw=3, marker = :o, ms = 6, label = L"s_1", c=3)
for m = 2:length(X)
    xp = range(X[m-1], X[m], length=20)
    plot!(xp, p2.(xp, f, X[m-1], Xm[m-1], X[m]), lw=3, label = "", c=2)
end
plot!(Xm, f.(Xm), m=:square, ms=6, c=2, lw=0, label = L"s_2")
plt

**Theorem:** Let $x_m$ be equispaced with spacing $h = x_m - x_{m-1}$ and let $s_N$ be the piecewise nodal interpolant of $f$ with equispaced nodes on each sub-interval, then 
$$ 
    \|f - s_N\|_\infty \leq \frac{(h/2)^{N+1}}{(N+1)!} \| f^{(N+1)} \|_\infty
$$

**Note:** For fixed $N$ but $M \to \infty$ ($h \to 0$) this is called *algebraic* convergence and should be contrasted with *exponential* convergence. More on this at the end of the course when we return to global polynomial approximation and spectral methods.

## Summary Approximation and Interpolation

* approximation of general functions by "simple functions"
* polynomial approximation, error vs smoothness!
* interpolation is a simple way for constructive approximation
* equispaced interpolation is *in general* unstable
* Chebyshev interpolation is not optimal but simple and close to optimal
* alternatively, use splines, i.e. control error through grid size / mesh size
* more complex: balance degree and mesh size, i.e. $hp$ approximation

### Further reading

* We may return to polynomial approximation in the last part of the course, e.g. how good is the best-approximation? How close is the Chebyshev interpolant to the best approximation? 
* There are many variants of spline interpolation, a popular class are **B-splines**
* Finite elements are the generalisation of splines to dimension > 1.

## Numerical Integration / Quadrature 

**Motivation:** Suppose we wish to numerically evaluate an integral 
$$ 
 I =  \int_a^b f(x) \,dx 
$$

A far-reaching idea is to approximate $f \approx p_N$ with a  polynomial and therefore approximate the integral 
$$ 
 I \approx I_N := \int_{a}^b p_N(x) \, dx
$$ 
Because $p_N$ is a polynomial, $I_N$ can be evaluated analytically. Applying this idea in a "piecewise" fashion leads to so-called *composite quadrature rules*.

**Topics:** midpoint rule, trapezoidal rule, Simpson rule, Newton-Cotes quadrature, Clenshaw-Curtis quadrature, Gauss quadrature

## Trapezoidal rule 

In [ ]:
f = x -> x/(1+10*x^2)
MATH405.illustrate_trapezoidal(f, 8)

Choose a partition $a = x_0 < x_1 < \dots, x_M = b$ then the *trapezoidal  rule* is 
$$ 
  I_N := \sum_{m = 1}^M \frac{x_m-x_{m-1}}{2} \big(f(x_{m-1}) + f(x_m)\big)
$$

### Error analysis

Suppose that $x_m = (b-a) / M$, $m = 0, \dots, M$ and $h = (b-a)/M = x_m - x_{m-1}$, then we know that the error in the linear interpolant is
$$
    \| f - s_1 \|_{\infty} \leq \frac{h^2}{8} \| f'' \|_{\infty} 
$$

and we immediately obtain 
$$
    \bigg| \int_a^b f - s_1 \,dx \bigg| \leq (b-a) \| f - s_1 \|_\infty \leq \frac{ (b-a) h^2}{8} \|f''\|_\infty.
$$

But we can do a little better if we remember that $f - s_1 = \frac{1}{2} (0-x) (h-x) f''(\xi)$. Therefore: 
$$ 
     \bigg|\int_0^h f - s_1 \,dx \bigg| 
     \leq \frac{\|f''\|_\infty}{2} \int_0^h | x (h-x) | \,dx
     \leq \frac{h^3 \|f''\|_\infty}{12}
$$

**Proposition:** The error for the composite trapezoidal rule is bounded by 
$$ 
  |I - I_M| \leq \frac{ (b-a) h^2}{12} \|f''\|_\infty.
$$

In [ ]:
trapezoidal(f, N) = ( 0.5/N * (f(0.0) + f(1.0)) + sum(f(n/N) for n=1:N-1)/N )

f = x -> x/(1+10*x^2)
If = log(11) / 20 # CHECK THIS!!
NN = [4, 8, 16, 32, 64, 128]
If_N = [ trapezoidal(f, N) for N in NN ]

plot(NN, abs.(If_N .- If), lw=2, m=:o, ms=6, label = L"|I - I_N|", 
    xaxis = :log, yaxis = :log, size = (600, 300), 
    title = "Convergence Trapezoidal rule")
plot!(NN[2:5], (1/12) * NN[2:5].^(-2), lw=2, ls=:dash, c=:black, label =  L"\sim N^{-2}")

### Midpoint rule

Use piecewise constant interpolation at the interval midpoints: Choose a partition $a = x_0 < x_1 < \dots, x_M = b$ then the *mid-point rule* is 
$$ 
  I_N := \sum_{m = 1}^M (x_m-x_{m-1}) f\big( 0.5 (x_{m-1}+x_m) \big)
$$

In [ ]:
f = x -> x/(1+10*x^2)
MATH405.illustrate_midpoint(f, 8)

**Challenge:** Guess the order of approximation?
$$ 
  I_M := \sum_{m = 1}^M (x_m-x_{m-1}) f\big( 0.5 (x_{m-1}+x_m) \big)
$$

In [ ]:
midpoint(f, N) = sum( f(x) for x in range(0.5/N, 1-0.5/N, length=N) ) / N

If = log(11) / 20 # CHECK THIS
If_mid = [ midpoint(f, N) for N in [4, 8, 16, 32, 64, 128] ]

plot(NN, abs.(If_N .- If), lw=2, m=:o, ms=6, label = "trapezoidal", 
    xaxis = :log, yaxis = :log, size = (600, 300), 
    title = "Convergence Midpoint Rule")
plot!(NN, abs.(If_mid .- If), lw=2, m=:o, ms=6, label = "midpoint")
plot!(NN[2:5], 0.2*NN[2:5].^(-2), lw=2, ls=:dash, c=:black, label =  L"\sim N^{-2}")

**Analysis of the mid-point rule:**  Piecewise constant approximation suggests that the quadrature error should be $O(h) = O(N^{-1})$ but we observe $O(h^2) = O(N^{-2})$. To explain this we can make a more careful Taylor expansion: 

$$
  f(x) = f(h/2) + f'(h/2) (x - h/2) + f''(\xi) (x-h/2)^2/2 
$$

$$
\begin{aligned}
  \int_0^h f(x) dx &= h f(h/2) + f'(h/2) \underset{=0}{\underbrace{\int_0^h (x-h/2) \,dx}} + {\rm remainder} \\ 
  |I - I_h| &\leq \frac{\|f''\|_\infty}{2} \int_0^h (x - h/2)^2 dx 
      = \frac{\|f''\|_\infty h^3}{24}
\end{aligned}
$$

Now we can argue as in the analysis of the trapezoidal rule to prove the following result: 

**Proposition:** Let $I_N$ denote the composite mid-point rule, then 
$$ 
   |I - I_N| \leq \frac{(b-a) h^2}{24}  \|f''\|_{\infty}.
$$

### Newton-Cotes Quadrature 

* Take $x_0, \dots, x_N$ equi-spaced, and $p$ the nodal interpolant of $f$, then the *open Newton-Cotes formula is*
$$
  \int_{x_0}^{x_N} f(x)\,dx \approx \int_{x_0}^{x_N} p(x)\,dx
  = (x_N-x_0) \sum_{n = 0}^N w_n f(x_n)
$$
(e.g. trapezoidal rule with $N = 1$

* Take $q$ the nodal interpolant at $x_1, \dots, x_{N-1}$ then the *closed Newton-Cotes formula* is 
$$
  \int_{x_0}^{x_N} f(x)\,dx \approx \int_{x_0}^{x_N} q(x)\,dx
   = (x_N - x_0) \sum_{ n = 1}^{N-1} w_n f(x_n).
$$
(e.g. mid-point rule with $N = 2$)


But remember that interpolation at equi-spaced points is unstable so don't let $N \to \infty$!!! For small or moderate $N$, these can be decent quadrature rules but they are rarely used for higher degrees where much better choices are available.!

### Simpson Rule 

E.g. the closed Newton-Cotes formula with $N=3$ is the Simpson rule 
$$ 
    I = \int_0^h f(x) dx \approx I_h = h  \Big(\frac{1}{6} f(0) + \frac{4}{6} f(h/2) + \frac{1}{6} f(h) \Big)
$$
with error 
$$ 
   |I - I_h| \leq \frac{ \|f^{(iv)} \|_\infty h^5}{2280}
$$

Determine weights: $p = $ quadratic interpolant of $f$ at nodes $0, h/2, h$
$$\begin{aligned}
   I &= \int_0^h f(x) dx \approx \int_0^h p(x) dx =: I_h \\
   p(x) &= f(0) L_0(x) + f(h/2) L_1(x) + f(h) L_2(x) \\ 
   I_h &=  \int_0^h p(x) dx = f(0) \int_0^h L_0(x) dx + f(h/2) \int_0^h L_1(x) dx + \dots
\end{aligned}$$

### General interpolatory quadrature rules

Take general interpolation points $x_0 < \dots, x_N$ and $p(x)$ the nodal interpolant of $f$ and approximate
$$
   I = \int_{x_0}^{x_N} f(x)\,dx \approx I_N := \int_{x_0}^{x_N} p(x) \,dx 
$$
then we have 
$$ 
I_N =  (x_N-x_0) \sum_{n = 0}^N f(x_n)
    \underset{=: w_n}{ \underbrace{ \frac{\int_{x_0}^{x_N} L_n(x) \,dx}{x_N-x_0} } } 
$$


e.g. Clenshaw-Curtis Quadrature : use Chebyshev nodes

## Summary Quadrature 

* Approximate $\int f$ by $\int p$ where $p$ is a polynomial interpolant
* increase polynomial degree or decrease mesh-size (splines!) for convergence
* close connection to polynomial and and spline approximation

### Further reading

* Clenshaw-Curtis quadrature
* Gauss quadrature 
* Adaptive Quadrature 
* $hp$ refinement